# Bids EEG

To create a BIDS directory for EEG, we will rely on the mne and mne_bids python packages. They will do most of the work for us.  
If not already installed, install mne and mne_bids packages.
```
pip3 install --user mne mne_bids
```

Now we are going to import our packages. In addition to mne and mne_bids, we will need glob, os, numpy, and the thalpy base.  
_You need to add the thalpy path in order to import its packages. If you haven't done that, check out the thalpy notebook explaining this._

In [1]:
import mne
from mne_bids import BIDSPath, write_raw_bids
import glob
import os
from thalpy import base
import numpy as np

Since we are creating a BIDS directory from our raw EEG data, we will need two main inputs. The directory holding the raw EEG files and the target BIDS directory (this should already be created, make it if you haven't).

## Biosemi Files - Our EEG Subjects

Any subjects run in our own lab will be in biosemi format. The extension of the raw EEG files are .bdf
For this example, we will be going through the process of creating the ThalHi EEG BIDS, but this process is the same for any EEG in Biosemi format. Let's create variables to store our raw eeg and target bids directories.  
  
A full example script can be found at /mnt/nfs/lss/lss_kahwang_hpc/scripts/thalhi/eeg/biosemi_to_bids.py

In [ ]:
raw_eeg_dir = '/mnt/cifs/rdss/rdss_kahwang/ThalHi_data/EEG_data/eeg_raw/'
bids_dir = '/data/backed_up/shared/ThalHi_EEG/BIDS'

For Biosemi format, the process of getting events from the EEG is a little more involved than other formats. You will need to define your triggers and their IDs in a dictionary.

In [ ]:
EVENT_IDS = {'faceProbe_trig': 151, 'sceneProbe_trig': 153, 'subResp_trig': 157, 'subNonResp_trig': 155,
             'IDS_trig': 101, 'Stay_trig': 105, 'EDS_trig': 103, 'ITI': 159, 'biosemiTrig_1': 65536, 'biosemiTrig_2': 65790}

Okay, now let's go those bdf files using glob.

In [ ]:
os.chdir(raw_eeg_dir)
bdf_files = glob.glob('*.bdf')

Now that we have our files, we can loop through each one and create the neccesaary bids files for it. If you are adding subjects or sessions to a BIDS directory that already extists you are going to want to check if the subject/session already exists in the BIDS directory. You can use a help function from the thalpy base function to pull out the subject id from the bdf file.

In [ ]:
for bdf_file in bdf_files:
    subject = base.parse_sub_from_file(bdf_file, prefix='sub_')
        if 'sub-' + subject in os.listdir(bids_dir):
            continue

For purposes of clarity and brevity, we are just going to look at processing one bdf file. Normally, you would loop through each one as shown above. Now that we have our raw bdf file, we will extract the events, set the bids path, and then write the new files into the BIDs directory. First, let's read in the raw bdf into mne format.

In [ ]:
raw = mne.io.read_raw_bdf(bdf_file)

To create events, we will use mne.find_events to pull events from the raw mne data. (You may have to set shortest_event to 1 here, if you get errors when reading the events).  
Then, we will go through and include only events found in our previously defined EVENT_IDS dictionary and return all eligible events in a numpy array.

In [ ]:
 def generate_events(raw):
    events_data = mne.find_events(raw, shortest_event=1)

    eligible_events = set(EVENT_IDS.values())
    ev_data_excludeIneligibleEvents = []
    print(events_data)
    for ev in events_data:
        # print(ev)
        if ev[2] in eligible_events:
            ev_data_excludeIneligibleEvents.append(ev)

    return np.asarray(ev_data_excludeIneligibleEvents)
 
 event_data = generate_events(raw)

We will use the BIDsPath object, passing in various data like our subject id, task, datatype as 'eeg', and the root as our target bids_dir. Check out the documentation for other parameters you may need to set.

BIDsPath documentation: https://mne.tools/mne-bids/dev/generated/mne_bids.BIDSPath.html

In [ ]:
bids_path = BIDSPath(subject=subject, task='ThalHi',  datatype='eeg',
                         root=bids_dir)

Finally, we will write the raw bids files to our target EEG. We will call the write_raw_bids funciton and pass our raw data, bids_path, events_data, and EVENT_IDs.

write_raw_bids docs: https://mne.tools/mne-bids/stable/generated/mne_bids.write_raw_bids.html

In [ ]:
write_raw_bids(raw, bids_path=bids_path, events_data=event_data, event_id=EVENT_IDS)

## Brainvision Files - Hospital Subjects

Subjects who undergo EEG at the hospital will have raw EEG data in brainvision format. The process will differ only slight in getting the files and generating events. I will only explain the differences as the rest of the process is the same.  
Example script: /mnt/nfs/lss/lss_kahwang_hpc/scripts/thalhi/eeg/brainvision_to_bids.py

The brainvision files you need have the .vhdr extension. As before, glob to get all the files.

In [ ]:
os.chdir(raw_eeg_dir)
bdf_files = glob.glob("*.vhdr")

The only other difference is generating events. The Brainvision events are easier to pull out. Simply use, the events_from_annotations funciton and pass the raw data. This will return the events data and event ids.

In [ ]:
events_data, events_id = mne.events_from_annotations(raw)